In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost
from xgboost import XGBRegressor

In [2]:
region_dict = {
    "1": "BSk",
    "9": "Dfb",
    "4": "Cfa",
    "7": "Csb",
    "8": "Dfa",
    "3": "BWk",
    "10": "Dfc",
    "2": "BWh",
    "6": "Csa",
    "11": "Dsb",
    "0": "BSh",
    "5": "Cfb",
    "12": "Dsc",
    "13": "Dwa",
    "14": "Dwb"
}

In [3]:
features = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3', 
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5',
 'diff_pres_1_first',
 'diff_pres_1_min',
 'diff_pres_1_max',
 'range_pres_1',
 'scale_pres_1',
 'diff_ccsm3_month_1_first',
 'diff_ccsm3_month_1_min',
 'diff_ccsm3_month_1_max',
 'range_ccsm3_month_1',
 'scale_ccsm3_month_1',
 'diff_sst_1_month_1_first',
 'diff_sst_1_month_1_min',
 'diff_sst_1_month_1_max',
 'range_sst_1_month_1',
 'scale_sst_1_month_1',
 'sst_1_lag_1',
 'sst_5_lag_1',
 'sst_4_lag_1',
 'sst_1_lag_1_season',
 'sst_5_lag_1_season',
 'sst_4_lag_1_season']

target=["contest-tmp2m-14d__tmp2m"]

slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_500 = 'contest-wind-h500-14d__wind-hgt-500'
sst_1_col = 'sst-2010-1'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'
group_cols = ['startdate']

In [4]:
df = pd.read_csv('./df_fe_time_3_pas.csv')

In [7]:

    
sst_features_5 = []
for i in [1, 5, 3, 4]:
    sst_features_5 = sst_features_5 + [f'sst_{i}_neigh_5_mean', f'sst_{i}_neigh_5_min', f'sst_{i}_neigh_5_max']

sst_features_5_diag_11 = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_11 = sst_features_5_diag_11 + [f'sst_{i}_neigh_5_diag_11_mean', f'sst_{i}_neigh_5_diag_11_min', f'sst_{i}_neigh_5_diag_11_max']


sst_features_5_diag_01 = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_01 = sst_features_5_diag_01 + [f'sst_{i}_neigh_5_diag_01_mean', f'sst_{i}_neigh_5_diag_01_min', f'sst_{i}_neigh_5_diag_01_max']

sst_features_5_diag_10 = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_10 = sst_features_5_diag_10 + [f'sst_{i}_neigh_5_diag_10_mean', f'sst_{i}_neigh_5_diag_10_min', f'sst_{i}_neigh_5_diag_10_max']

sst_features_5_diag_00 = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_00 = sst_features_5_diag_00 + [f'sst_{i}_neigh_5_diag_00_mean', f'sst_{i}_neigh_5_diag_00_min', f'sst_{i}_neigh_5_diag_00_max']


sst_features_5_diag_pos = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_pos = sst_features_5_diag_pos + [f'sst_{i}_neigh_5_diag_pos_mean', f'sst_{i}_neigh_5_diag_pos_min', f'sst_{i}_neigh_5_diag_pos_max']

sst_features_5 = []
for i in [1, 5, 3, 4]:
    sst_features_5 = sst_features_5 + [f'sst_{i}_neigh_5_mean', f'sst_{i}_neigh_5_min', f'sst_{i}_neigh_5_max']

sst_features_5_lon_neg = []
for i in [1, 5, 3, 4]:
    sst_features_5_lon_neg = sst_features_5_lon_neg + [f'sst_{i}_neigh_5_lon_neg_mean', f'sst_{i}_neigh_5_lon_neg_min', f'sst_{i}_neigh_5_lon_neg_max']

sst_features_5_lat_neg = []
for i in [1, 5, 3, 4]:
    sst_features_5_lat_neg = sst_features_5_lat_neg + [f'sst_{i}_neigh_5_lat_neg_mean', f'sst_{i}_neigh_5_lat_neg_min', f'sst_{i}_neigh_5_lat_neg_max']


sst_features_5_lat_pos = []
for i in [1, 5, 3, 4]:
    sst_features_5_lat_pos = sst_features_5_lat_pos + [f'sst_{i}_neigh_5_lat_pos_mean', f'sst_{i}_neigh_5_lat_pos_min', f'sst_{i}_neigh_5_lat_pos_max']

sst_features_5_lon_pos = []
for i in [1, 5, 3, 4]:
    sst_features_5_lon_pos = sst_features_5_lon_pos + [f'sst_{i}_neigh_5_lon_pos_mean', f'sst_{i}_neigh_5_lon_pos_min', f'sst_{i}_neigh_5_lon_pos_max']

sst_features_5_diag_pos = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_pos = sst_features_5_diag_pos + [f'sst_{i}_neigh_5_diag_pos_mean', f'sst_{i}_neigh_5_diag_pos_min', f'sst_{i}_neigh_5_diag_pos_max']

sst_features_5_diag_neg1 = []
for i in [1, 5, 3, 4]:
    sst_features_5_diag_neg1 = sst_features_5_diag_neg1 + [f'sst_{i}_neigh_5_diag_neg1_mean', f'sst_{i}_neigh_5_diag_neg1_min', f'sst_{i}_neigh_5_diag_neg1_max']

sst_features_time_3_pos = []
for i in [1, 5, 3, 4]:
    sst_features_time_3_pos = sst_features_time_3_pos + [f'sst_{i}_time_3_pas_mean', f'sst_{i}_time_3_pas_min', f'sst_{i}_time_3_pas_max']

sst_features_time_5 = []
for i in [1, 5, 3, 4]:
    sst_features_time_5 = sst_features_time_5 + [f'sst_{i}_time_5_mean', f'sst_{i}_time_5_min', f'sst_{i}_time_5_max']

sst_features_time_3_pas= []
for i in [1, 5, 3, 4]:
    sst_features_time_3_pas= sst_features_time_3_pas+ [f'sst_{i}_time_3_pas_mean', f'sst_{i}_time_3_pas_min', f'sst_{i}_time_3_pas_max']

sst_features_time_7 = []
for i in [1, 5, 3, 4]:
    sst_features_time_7 = sst_features_time_7 + [f'sst_{i}_time_7_mean', f'sst_{i}_time_7_min', f'sst_{i}_time_7_max']


sst_features_time_14 = []
for i in [1, 5, 3, 4]:
    sst_features_time_14 = sst_features_time_14 + [f'sst_{i}_time_14_mean', f'sst_{i}_time_14_min', f'sst_{i}_time_14_max']

sst_features_time_9 = []
for i in [1, 5, 3, 4]:
    sst_features_time_9 = sst_features_time_9 + [f'sst_{i}_time_9_mean', f'sst_{i}_time_9_min', f'sst_{i}_time_9_max']


sst_features_time_11 = []
for i in [1, 5, 3, 4]:
    sst_features_time_11 = sst_features_time_11 + [f'sst_{i}_time_11_mean', f'sst_{i}_time_11_min', f'sst_{i}_time_11_max']

sst_features_11 = []
for i in [1, 5, 3, 4]:
    sst_features_11 = sst_features_11 + [f'sst_{i}_neigh_11_mean', f'sst_{i}_neigh_11_min', f'sst_{i}_neigh_11_max']

sst_features_3 = []
for i in [1, 5, 3, 4]:
    sst_features_3 = sst_features_3 + [f'sst_{i}_neigh_3_mean', f'sst_{i}_neigh_3_min', f'sst_{i}_neigh_3_max']

sst_features_7 = []
for i in [1, 5, 3, 4]:
    sst_features_7 = sst_features_7 + [f'sst_{i}_neigh_7_mean', f'sst_{i}_neigh_7_min', f'sst_{i}_neigh_7_max']

sst_features_5_lat = []
for i in [1, 5, 3, 4]:
    sst_features_5_lat = sst_features_5_lat + [f'sst_{i}_neigh_5_lat_mean', f'sst_{i}_neigh_5_lat_min', f'sst_{i}_neigh_5_lat_max']

sst_features_5_lon = []
for i in [1, 5, 3, 4]:
    sst_features_5_lon = sst_features_5_lon + [f'sst_{i}_neigh_5_lon_mean', f'sst_{i}_neigh_5_lon_min', f'sst_{i}_neigh_5_lon_max']

sst_features_9_lon = []
for i in [1, 5, 3, 4]:
    sst_features_9_lon = sst_features_9_lon + [f'sst_{i}_neigh_9_lon_mean', f'sst_{i}_neigh_9_lon_min', f'sst_{i}_neigh_9_lon_max']

sst_features_11_lon = []
for i in [1, 5, 3, 4]:
    sst_features_11_lon = sst_features_11_lon + [f'sst_{i}_neigh_11_lon_mean', f'sst_{i}_neigh_11_lon_min', f'sst_{i}_neigh_11_lon_max']

sst_features_15_lon = []
for i in [1, 5, 3, 4]:
    sst_features_15_lon = sst_features_15_lon + [f'sst_{i}_neigh_15_lon_mean', f'sst_{i}_neigh_15_lon_min', f'sst_{i}_neigh_15_lon_max']

sst_features_11_lat = []
for i in [1, 5, 3, 4]:
    sst_features_11_lat = sst_features_11_lat + [f'sst_{i}_neigh_11_lat_mean', f'sst_{i}_neigh_11_lat_min', f'sst_{i}_neigh_11_lat_max']

sst_features_9_lat = []
for i in [1, 5, 3, 4]:
    sst_features_9_lat = sst_features_9_lat + [f'sst_{i}_neigh_9_lat_mean', f'sst_{i}_neigh_9_lat_min', f'sst_{i}_neigh_9_lat_max']
    
sst_features_9 = []
for i in [1, 5, 3, 4]:
    sst_features_9 = sst_features_9 + [f'sst_{i}_neigh_9_mean', f'sst_{i}_neigh_9_min', f'sst_{i}_neigh_9_max']

sst_features_13 = []
for i in [1, 5, 3, 4]:
    sst_features_13 = sst_features_13 + [f'sst_{i}_neigh_13_mean', f'sst_{i}_neigh_13_min', f'sst_{i}_neigh_13_max']


poten_feature_5 = ['poten_neigh_5_mean', 'poten_neigh_5_min', 'poten_neigh_5_max']
poten_feature_time_5 = ['poten_time_5_mean', 'poten_time_5_min', 'poten_time_5_max']
poten_feature_time_5_pos = ['poten_time_5_pos_mean', 'poten_time_5_pos_min', 'poten_time_5_pos_max']
poten_feature_time_7 = ['poten_time_7_mean', 'poten_time_7_min', 'poten_time_7_max']
poten_feature_time_11 = ['poten_time_11_mean', 'poten_time_11_min', 'poten_time_11_max']
poten_feature_time_9 = ['poten_time_9_mean', 'poten_time_9_min', 'poten_time_9_max']

poten_feature_3 = ['poten_neigh_3_mean', 'poten_neigh_3_min', 'poten_neigh_3_max']
poten_feature_7 = ['poten_neigh_7_mean', 'poten_neigh_7_min', 'poten_neigh_7_max']
poten_feature_9 = ['poten_neigh_9_mean', 'poten_neigh_9_min', 'poten_neigh_9_max']
poten_feature_11 = ['poten_neigh_11_mean', 'poten_neigh_11_min', 'poten_neigh_11_max']
poten_feature_13 = ['poten_neigh_13_mean', 'poten_neigh_13_min', 'poten_neigh_13_max']



rhum_feature_5 = ['rhum_neigh_5_mean', 'rhum_neigh_5_min', 'rhum_neigh_5_max']
rhum_feature_5_diag_pos = ['rhum_neigh_5_diag_pos_mean', 'rhum_neigh_5_diag_pos_min', 'rhum_neigh_5_diag_pos_max']
rhum_feature_5_diag_neg1 = ['rhum_neigh_5_diag_neg1_mean', 'rhum_neigh_5_diag_neg1_min', 'rhum_neigh_5_diag_neg1_max']

rhum_feature_5_lat = ['rhum_neigh_5_lat_mean', 'rhum_neigh_5_lat_min', 'rhum_neigh_5_lat_max']
rhum_feature_5_lat_neg = ['rhum_neigh_5_lat_neg_mean', 'rhum_neigh_5_lat_neg_min', 'rhum_neigh_5_lat_neg_max']
rhum_feature_5_lon_neg = ['rhum_neigh_5_lon_neg_mean', 'rhum_neigh_5_lon_neg_min', 'rhum_neigh_5_lon_neg_max']
rhum_feature_5_lat_pos = ['rhum_neigh_5_lat_pos_mean', 'rhum_neigh_5_lat_pos_min', 'rhum_neigh_5_lat_pos_max']
rhum_feature_5_lon_pos = ['rhum_neigh_5_lon_pos_mean', 'rhum_neigh_5_lon_pos_min', 'rhum_neigh_5_lon_pos_max']
rhum_feature_5_lon = ['rhum_neigh_5_lon_mean', 'rhum_neigh_5_lon_min', 'rhum_neigh_5_lon_max']
rhum_feature_9_lon = ['rhum_neigh_9_lon_mean', 'rhum_neigh_9_lon_min', 'rhum_neigh_9_lon_max']
rhum_feature_11_lon = ['rhum_neigh_11_lon_mean', 'rhum_neigh_11_lon_min', 'rhum_neigh_11_lon_max']
rhum_feature_15_lon = ['rhum_neigh_15_lon_mean', 'rhum_neigh_15_lon_min', 'rhum_neigh_15_lon_max']
rhum_feature_11_lat = ['rhum_neigh_11_lat_mean', 'rhum_neigh_11_lat_min', 'rhum_neigh_11_lat_max']
rhum_feature_9_lat = ['rhum_neigh_9_lat_mean', 'rhum_neigh_9_lat_min', 'rhum_neigh_9_lat_max']
rhum_feature_time_3 = ['rhum_time_3_mean', 'rhum_time_3_min', 'rhum_time_3_max']
rhum_feature_time_3_pas = ['rhum_time_3_pas_mean', 'rhum_time_3_pas_min', 'rhum_time_3_pas_max']

rhum_feature_time_5 = ['rhum_time_5_mean', 'rhum_time_5_min', 'rhum_time_5_max']
rhum_feature_time_5_pos = ['rhum_time_5_pos_mean', 'rhum_time_5_pos_min', 'rhum_time_5_pos_max']
rhum_feature_time_7 = ['rhum_time_7_mean', 'rhum_time_7_min', 'rhum_time_7_max']
rhum_feature_time_11 = ['rhum_time_11_mean', 'rhum_time_11_min', 'rhum_time_11_max']
rhum_feature_time_7_mid = ['rhum_time_7_mid_mean', 'rhum_time_7_mid_min', 'rhum_time_7_mid_max']
rhum_feature_time_14 = ['rhum_time_14_mean', 'rhum_time_14_min', 'rhum_time_14_max']
rhum_feature_time_21 = ['rhum_time_21_mean', 'rhum_time_21_min', 'rhum_time_21_max']
rhum_feature_time_28 = ['rhum_time_28_mean', 'rhum_time_28_min', 'rhum_time_28_max']
rhum_feature_time_9 = ['rhum_time_9_mean', 'rhum_time_9_min', 'rhum_time_9_max']

rhum_feature_3 = ['rhum_neigh_3_mean', 'rhum_neigh_3_min', 'rhum_neigh_3_max']
rhum_feature_7 = ['rhum_neigh_7_mean', 'rhum_neigh_7_min', 'rhum_neigh_7_max']
rhum_feature_9 = ['rhum_neigh_9_mean', 'rhum_neigh_9_min', 'rhum_neigh_9_max']
rhum_feature_11 = ['rhum_neigh_11_mean', 'rhum_neigh_11_min', 'rhum_neigh_11_max']
rhum_feature_13 = ['rhum_neigh_13_mean', 'rhum_neigh_13_min', 'rhum_neigh_13_max']


slp_feature_5 = ['slp_neigh_5_mean', 'slp_neigh_5_min', 'slp_neigh_5_max']
slp_feature_5_diag_pos = ['slp_neigh_5_diag_pos_mean', 'slp_neigh_5_diag_pos_min', 'slp_neigh_5_diag_pos_max']
slp_feature_5_diag_neg1 = ['slp_neigh_5_diag_neg1_mean', 'slp_neigh_5_diag_neg1_min', 'slp_neigh_5_diag_neg1_max']
slp_feature_5_lat = ['slp_neigh_5_lat_mean', 'slp_neigh_5_lat_min', 'slp_neigh_5_lat_max']
slp_feature_5_lat_pos = ['slp_neigh_5_lat_pos_mean', 'slp_neigh_5_lat_pos_min', 'slp_neigh_5_lat_pos_max']
slp_feature_5_lat_neg = ['slp_neigh_5_lat_neg_mean', 'slp_neigh_5_lat_neg_min', 'slp_neigh_5_lat_neg_max']
slp_feature_5_lon_pos = ['slp_neigh_5_lon_pos_mean', 'slp_neigh_5_lon_pos_min', 'slp_neigh_5_lon_pos_max']
slp_feature_5_lon_neg = ['slp_neigh_5_lon_neg_mean', 'slp_neigh_5_lon_neg_min', 'slp_neigh_5_lon_neg_max']
slp_feature_5_lon = ['slp_neigh_5_lon_mean', 'slp_neigh_5_lon_min', 'slp_neigh_5_lon_max']
slp_feature_9_lon = ['slp_neigh_9_lon_mean', 'slp_neigh_9_lon_min', 'slp_neigh_9_lon_max']
slp_feature_11_lon = ['slp_neigh_11_lon_mean', 'slp_neigh_11_lon_min', 'slp_neigh_11_lon_max']
slp_feature_15_lon = ['slp_neigh_15_lon_mean', 'slp_neigh_15_lon_min', 'slp_neigh_15_lon_max']
slp_feature_11_lat = ['slp_neigh_11_lat_mean', 'slp_neigh_11_lat_min', 'slp_neigh_11_lat_max']
slp_feature_9_lat = ['slp_neigh_9_lat_mean', 'slp_neigh_9_lat_min', 'slp_neigh_9_lat_max']
slp_feature_time_5 = ['slp_time_5_mean', 'slp_time_5_min', 'slp_time_5_max']
slp_feature_time_3_pas = ['slp_time_3_pas_mean', 'slp_time_3_pas_min', 'slp_time_3_pas_max']

slp_feature_time_5_pos = ['slp_time_5_pos_mean', 'slp_time_5_pos_min', 'slp_time_5_pos_max']
slp_feature_time_7 = ['slp_time_7_mean', 'slp_time_7_min', 'slp_time_7_max']
slp_feature_time_11 = ['slp_time_11_mean', 'slp_time_11_min', 'slp_time_11_max']
slp_feature_time_14 = ['slp_time_14_mean', 'slp_time_14_min', 'slp_time_14_max']
slp_feature_time_9 = ['slp_time_9_mean', 'slp_time_9_min', 'slp_time_9_max']

slp_feature_3 = ['slp_neigh_3_mean', 'slp_neigh_3_min', 'slp_neigh_3_max']
slp_feature_7 = ['slp_neigh_7_mean', 'slp_neigh_7_min', 'slp_neigh_7_max']
slp_feature_9 = ['slp_neigh_9_mean', 'slp_neigh_9_min', 'slp_neigh_9_max']
slp_feature_11 = ['slp_neigh_11_mean', 'slp_neigh_11_min', 'slp_neigh_11_max']
slp_feature_13 = ['slp_neigh_13_mean', 'slp_neigh_13_min', 'slp_neigh_13_max']


precip_feature_5 = ['precip_neigh_5_mean', 'precip_neigh_5_min', 'precip_neigh_5_max']
precip_feature_5_diag_pos = ['precip_neigh_5_diag_pos_mean', 'precip_neigh_5_diag_pos_min', 'precip_neigh_5_diag_pos_max']
precip_feature_5_diag_neg1 = ['precip_neigh_5_diag_neg1_mean', 'precip_neigh_5_diag_neg1_min', 'precip_neigh_5_diag_neg1_max']
precip_feature_5_lat = ['precip_neigh_5_lat_mean', 'precip_neigh_5_lat_min', 'precip_neigh_5_lat_max']
precip_feature_5_lat_neg = ['precip_neigh_5_lat_neg_mean', 'precip_neigh_5_lat_neg_min', 'precip_neigh_5_lat_neg_max']
precip_feature_5_lat_pos = ['precip_neigh_5_lat_pos_mean', 'precip_neigh_5_lat_pos_min', 'precip_neigh_5_lat_pos_max']
precip_feature_5_lon_pos = ['precip_neigh_5_lon_pos_mean', 'precip_neigh_5_lon_pos_min', 'precip_neigh_5_lon_pos_max']
precip_feature_5_lon_neg = ['precip_neigh_5_lon_neg_mean', 'precip_neigh_5_lon_neg_min', 'precip_neigh_5_lon_neg_max']
precip_feature_5_lat = ['precip_neigh_5_lat_mean', 'precip_neigh_5_lat_min', 'precip_neigh_5_lat_max']
precip_feature_5_lon = ['precip_neigh_5_lon_mean', 'precip_neigh_5_lon_min', 'precip_neigh_5_lon_max']
precip_feature_9_lon = ['precip_neigh_9_lon_mean', 'precip_neigh_9_lon_min', 'precip_neigh_9_lon_max']
precip_feature_11_lon = ['precip_neigh_11_lon_mean', 'precip_neigh_11_lon_min', 'precip_neigh_11_lon_max']
precip_feature_15_lon = ['precip_neigh_15_lon_mean', 'precip_neigh_15_lon_min', 'precip_neigh_15_lon_max']
precip_feature_11_lat = ['precip_neigh_11_lat_mean', 'precip_neigh_11_lat_min', 'precip_neigh_11_lat_max']
precip_feature_9_lat = ['precip_neigh_9_lat_mean', 'precip_neigh_9_lat_min', 'precip_neigh_9_lat_max']
precip_feature_time_3 = ['precip_time_3_mean', 'precip_time_3_min', 'precip_time_3_max']
precip_feature_time_3_pas = ['precip_time_3_pas_mean', 'precip_time_3_pas_min', 'precip_time_3_pas_max']

precip_feature_time_5 = ['precip_time_5_mean', 'precip_time_5_min', 'precip_time_5_max']
precip_feature_time_5_pos = ['precip_time_5_pos_mean', 'precip_time_5_pos_min', 'precip_time_5_pos_max']
precip_feature_time_7 = ['precip_time_7_mean', 'precip_time_7_min', 'precip_time_7_max']
precip_feature_time_11 = ['precip_time_11_mean', 'precip_time_11_min', 'precip_time_11_max']
precip_feature_time_7_mid = ['precip_time_7_mid_mean', 'precip_time_7_mid_min', 'precip_time_7_mid_max']
precip_feature_time_9 = ['precip_time_9_mean', 'precip_time_9_min', 'precip_time_9_max']
precip_feature_time_14 = ['precip_time_14_mean', 'precip_time_14_min', 'precip_time_14_max']
precip_feature_time_21 = ['precip_time_21_mean', 'precip_time_21_min', 'precip_time_21_max']
precip_feature_time_28 = ['precip_time_28_mean', 'precip_time_28_min', 'precip_time_28_max']

precip_feature_3 = ['precip_neigh_3_mean', 'precip_neigh_3_min', 'precip_neigh_3_max']
precip_feature_7 = ['precip_neigh_7_mean', 'precip_neigh_7_min', 'precip_neigh_7_max']
precip_feature_9 = ['precip_neigh_9_mean', 'precip_neigh_9_min', 'precip_neigh_9_max']
precip_feature_11 = ['precip_neigh_11_mean', 'precip_neigh_11_min', 'precip_neigh_11_max']
precip_feature_13 = ['precip_neigh_13_mean', 'precip_neigh_13_min', 'precip_neigh_13_max']


pres_feature_5 = ['pres_neigh_5_mean', 'pres_neigh_5_min', 'pres_neigh_5_max']
pres_feature_5_diag_pos = ['pres_neigh_5_diag_pos_mean', 'pres_neigh_5_diag_pos_min', 'pres_neigh_5_diag_pos_max']
pres_feature_5_diag_neg1 = ['pres_neigh_5_diag_neg1_mean', 'pres_neigh_5_diag_neg1_min', 'pres_neigh_5_diag_neg1_max']
pres_feature_5_lat = ['pres_neigh_5_lat_mean', 'pres_neigh_5_lat_min', 'pres_neigh_5_lat_max']
pres_feature_5_lat_neg = ['pres_neigh_5_lat_neg_mean', 'pres_neigh_5_lat_neg_min', 'pres_neigh_5_lat_neg_max']
pres_feature_5_lon_neg = ['pres_neigh_5_lon_neg_mean', 'pres_neigh_5_lon_neg_min', 'pres_neigh_5_lon_neg_max']
pres_feature_5_lat_pos = ['pres_neigh_5_lat_pos_mean', 'pres_neigh_5_lat_pos_min', 'pres_neigh_5_lat_pos_max']
pres_feature_5_lon_pos = ['pres_neigh_5_lon_pos_mean', 'pres_neigh_5_lon_pos_min', 'pres_neigh_5_lon_pos_max']
pres_feature_5_lon = ['pres_neigh_5_lon_mean', 'pres_neigh_5_lon_min', 'pres_neigh_5_lon_max']
pres_feature_9_lon = ['pres_neigh_9_lon_mean', 'pres_neigh_9_lon_min', 'pres_neigh_9_lon_max']
pres_feature_15_lon = ['pres_neigh_15_lon_mean', 'pres_neigh_15_lon_min', 'pres_neigh_15_lon_max']
pres_feature_11_lat = ['pres_neigh_11_lat_mean', 'pres_neigh_11_lat_min', 'pres_neigh_11_lat_max']
pres_feature_9_lat = ['pres_neigh_9_lat_mean', 'pres_neigh_9_lat_min', 'pres_neigh_9_lat_max']
pres_feature_time_3 = ['pres_time_3_mean', 'pres_time_3_min', 'pres_time_3_max']
pres_feature_time_3_pas = ['pres_time_3_pas_mean', 'pres_time_3_pas_min', 'pres_time_3_pas_max']
pres_feature_time_5 = ['pres_time_5_mean', 'pres_time_5_min', 'pres_time_5_max']
pres_feature_time_5_pos = ['pres_time_5_pos_mean', 'pres_time_5_pos_min', 'pres_time_5_pos_max']
pres_feature_time_7 = ['pres_time_7_mean', 'pres_time_7_min', 'pres_time_7_max']
pres_feature_time_11 = ['pres_time_11_mean', 'pres_time_11_min', 'pres_time_11_max']
pres_feature_time_7_mid = ['pres_time_7_mid_mean', 'pres_time_7_mid_min', 'pres_time_7_mid_max']
pres_feature_time_9 = ['pres_time_9_mean', 'pres_time_9_min', 'pres_time_9_max']
pres_feature_time_14 = ['pres_time_14_mean', 'pres_time_14_min', 'pres_time_14_max']
pres_feature_time_21 = ['pres_time_21_mean', 'pres_time_21_min', 'pres_time_21_max']
pres_feature_time_28 = ['pres_time_28_mean', 'pres_time_28_min', 'pres_time_28_max']

pres_feature_3 = ['pres_neigh_3_mean', 'pres_neigh_3_min', 'pres_neigh_3_max']
pres_feature_7 = ['pres_neigh_7_mean', 'pres_neigh_7_min', 'pres_neigh_7_max']
pres_feature_9 = ['pres_neigh_9_mean', 'pres_neigh_9_min', 'pres_neigh_9_max']
pres_feature_11 = ['pres_neigh_11_mean', 'pres_neigh_11_min', 'pres_neigh_11_max']
pres_feature_13 = ['pres_neigh_13_mean', 'pres_neigh_13_min', 'pres_neigh_13_max']



wind_feature_11 = ['wind_500_neigh_11_mean', 'wind_500_neigh_11_min', 'wind_500_neigh_11_max']
wind_feature_9 = ['wind_500_neigh_9_mean', 'wind_500_neigh_9_min', 'wind_500_neigh_9_max']
wind_feature_5 = ['wind_500_neigh_5_mean', 'wind_500_neigh_5_min', 'wind_500_neigh_5_max']
wind_feature_5_diag_pos = ['wind_500_neigh_5_diag_pos_mean', 'wind_500_neigh_5_diag_pos_min', 'wind_500_neigh_5_diag_pos_max']
wind_feature_5_diag_neg1 = ['wind_500_neigh_5_diag_neg1_mean', 'wind_500_neigh_5_diag_neg1_min', 'wind_500_neigh_5_diag_neg1_max']
wind_feature_5_lat = ['wind_500_neigh_5_lat_mean', 'wind_500_neigh_5_lat_min', 'wind_500_neigh_5_lat_max']
wind_feature_5_lat_pos = ['wind_500_neigh_5_lat_pos_mean', 'wind_500_neigh_5_lat_pos_min', 'wind_500_neigh_5_lat_pos_max']
wind_feature_5_lat_neg = ['wind_500_neigh_5_lat_neg_mean', 'wind_500_neigh_5_lat_neg_min', 'wind_500_neigh_5_lat_neg_max']
wind_feature_5_lon_pos = ['wind_500_neigh_5_lon_pos_mean', 'wind_500_neigh_5_lon_pos_min', 'wind_500_neigh_5_lon_pos_max']
wind_feature_5_lon_neg = ['wind_500_neigh_5_lon_neg_mean', 'wind_500_neigh_5_lon_neg_min', 'wind_500_neigh_5_lon_neg_max']
wind_feature_5_lon = ['wind_500_neigh_5_lon_mean', 'wind_500_neigh_5_lon_min', 'wind_500_neigh_5_lon_max']
wind_feature_9_lon = ['wind_500_neigh_9_lon_mean', 'wind_500_neigh_9_lon_min', 'wind_500_neigh_9_lon_max']
wind_feature_11_lon = ['wind_500_neigh_11_lon_mean', 'wind_500_neigh_11_lon_min', 'wind_500_neigh_11_lon_max']
wind_feature_15_lon = ['wind_500_neigh_15_lon_mean', 'wind_500_neigh_15_lon_min', 'wind_500_neigh_15_lon_max']
wind_feature_11_lat = ['wind_500_neigh_11_lat_mean', 'wind_500_neigh_11_lat_min', 'wind_500_neigh_11_lat_max']
wind_feature_9_lat = ['wind_500_neigh_9_lat_mean', 'wind_500_neigh_9_lat_min', 'wind_500_neigh_9_lat_max']
wind_feature_time_5 = ['wind_500_time_5_mean', 'wind_500_time_5_min', 'wind_500_time_5_max']
wind_feature_time_3_pas = ['wind_500_time_3_pas_mean', 'wind_500_time_3_pas_min', 'wind_500_time_3_pas_max']

wind_feature_time_5_pos = ['wind_500_time_5_pos_mean', 'wind_500_time_5_pos_min', 'wind_500_time_5_pos_max']
wind_feature_time_7 = ['wind_500_time_7_mean', 'wind_500_time_7_min', 'wind_500_time_7_max']
wind_feature_time_14 = ['wind_500_time_14_mean', 'wind_500_time_14_min', 'wind_500_time_14_max']
wind_feature_time_9 = ['wind_500_time_9_mean', 'wind_500_time_9_min', 'wind_500_time_9_max']
wind_feature_time_11 = ['wind_500_time_11_mean', 'wind_500_time_11_min', 'wind_500_time_11_max']
wind_feature_7 = ['wind_500_neigh_7_mean', 'wind_500_neigh_7_min', 'wind_500_neigh_7_max']
wind_feature_3 = ['wind_500_neigh_3_mean', 'wind_500_neigh_3_min', 'wind_500_neigh_3_max']
wind_feature_13 = ['wind_500_neigh_13_mean', 'wind_500_neigh_13_min', 'wind_500_neigh_13_max']

sst_features_time_5 = []
for i in [1, 5, 3, 4]:
    sst_features_time_5 = sst_features_time_5 + [f'sst_{i}_time_5_mean', f'sst_{i}_time_5_min', f'sst_{i}_time_5_max']

sst_features_time_7 = []
for i in [1, 5, 3, 4]:
    sst_features_time_7 = sst_features_time_7 + [f'sst_{i}_time_7_mean', f'sst_{i}_time_7_min', f'sst_{i}_time_7_max']


sst_features_time_14 = []
for i in [1, 5, 3, 4]:
    sst_features_time_14 = sst_features_time_14 + [f'sst_{i}_time_14_mean', f'sst_{i}_time_14_min', f'sst_{i}_time_14_max']

sst_features_time_9 = []
for i in [1, 5, 3, 4]:
    sst_features_time_9 = sst_features_time_9 + [f'sst_{i}_time_9_mean', f'sst_{i}_time_9_min', f'sst_{i}_time_9_max']


sst_features_time_11 = []
for i in [1, 5, 3, 4]:
    sst_features_time_11 = sst_features_time_11 + [f'sst_{i}_time_11_mean', f'sst_{i}_time_11_min', f'sst_{i}_time_11_max']

sst_features_11 = []
for i in [1, 5, 3, 4]:
    sst_features_11 = sst_features_11 + [f'sst_{i}_neigh_11_mean', f'sst_{i}_neigh_11_min', f'sst_{i}_neigh_11_max']

sst_features_3 = []
for i in [1, 5, 3, 4]:
    sst_features_3 = sst_features_3 + [f'sst_{i}_neigh_3_mean', f'sst_{i}_neigh_3_min', f'sst_{i}_neigh_3_max']

sst_features_7 = []
for i in [1, 5, 3, 4]:
    sst_features_7 = sst_features_7 + [f'sst_{i}_neigh_7_mean', f'sst_{i}_neigh_7_min', f'sst_{i}_neigh_7_max']

sst_features_5_lat = []
for i in [1, 5, 3, 4]:
    sst_features_5_lat = sst_features_5_lat + [f'sst_{i}_neigh_5_lat_mean', f'sst_{i}_neigh_5_lat_min', f'sst_{i}_neigh_5_lat_max']

sst_features_5_lon = []
for i in [1, 5, 3, 4]:
    sst_features_5_lon = sst_features_5_lon + [f'sst_{i}_neigh_5_lon_mean', f'sst_{i}_neigh_5_lon_min', f'sst_{i}_neigh_5_lon_max']

sst_features_9_lon = []
for i in [1, 5, 3, 4]:
    sst_features_9_lon = sst_features_9_lon + [f'sst_{i}_neigh_9_lon_mean', f'sst_{i}_neigh_9_lon_min', f'sst_{i}_neigh_9_lon_max']

sst_features_11_lon = []
for i in [1, 5, 3, 4]:
    sst_features_11_lon = sst_features_11_lon + [f'sst_{i}_neigh_11_lon_mean', f'sst_{i}_neigh_11_lon_min', f'sst_{i}_neigh_11_lon_max']

sst_features_15_lon = []
for i in [1, 5, 3, 4]:
    sst_features_15_lon = sst_features_15_lon + [f'sst_{i}_neigh_15_lon_mean', f'sst_{i}_neigh_15_lon_min', f'sst_{i}_neigh_15_lon_max']

sst_features_11_lat = []
for i in [1, 5, 3, 4]:
    sst_features_11_lat = sst_features_11_lat + [f'sst_{i}_neigh_11_lat_mean', f'sst_{i}_neigh_11_lat_min', f'sst_{i}_neigh_11_lat_max']

sst_features_9_lat = []
for i in [1, 5, 3, 4]:
    sst_features_9_lat = sst_features_9_lat + [f'sst_{i}_neigh_9_lat_mean', f'sst_{i}_neigh_9_lat_min', f'sst_{i}_neigh_9_lat_max']
    
sst_features_9 = []
for i in [1, 5, 3, 4]:
    sst_features_9 = sst_features_9 + [f'sst_{i}_neigh_9_mean', f'sst_{i}_neigh_9_min', f'sst_{i}_neigh_9_max']

sst_features_13 = []
for i in [1, 5, 3, 4]:
    sst_features_13 = sst_features_13 + [f'sst_{i}_neigh_13_mean', f'sst_{i}_neigh_13_min', f'sst_{i}_neigh_13_max']


poten_feature_5 = ['poten_neigh_5_mean', 'poten_neigh_5_min', 'poten_neigh_5_max']
poten_feature_time_5 = ['poten_time_5_mean', 'poten_time_5_min', 'poten_time_5_max']
poten_feature_time_7 = ['poten_time_7_mean', 'poten_time_7_min', 'poten_time_7_max']
poten_feature_time_11 = ['poten_time_11_mean', 'poten_time_11_min', 'poten_time_11_max']
poten_feature_time_9 = ['poten_time_9_mean', 'poten_time_9_min', 'poten_time_9_max']

poten_feature_3 = ['poten_neigh_3_mean', 'poten_neigh_3_min', 'poten_neigh_3_max']
poten_feature_7 = ['poten_neigh_7_mean', 'poten_neigh_7_min', 'poten_neigh_7_max']
poten_feature_9 = ['poten_neigh_9_mean', 'poten_neigh_9_min', 'poten_neigh_9_max']
poten_feature_11 = ['poten_neigh_11_mean', 'poten_neigh_11_min', 'poten_neigh_11_max']
poten_feature_13 = ['poten_neigh_13_mean', 'poten_neigh_13_min', 'poten_neigh_13_max']



rhum_feature_5 = ['rhum_neigh_5_mean', 'rhum_neigh_5_min', 'rhum_neigh_5_max']
rhum_feature_5_diag_11 = ['rhum_neigh_5_diag_11_mean', 'rhum_neigh_5_diag_11_min', 'rhum_neigh_5_diag_11_max']
rhum_feature_5_diag_10 = ['rhum_neigh_5_diag_10_mean', 'rhum_neigh_5_diag_10_min', 'rhum_neigh_5_diag_10_max']
rhum_feature_5_diag_01 = ['rhum_neigh_5_diag_01_mean', 'rhum_neigh_5_diag_01_min', 'rhum_neigh_5_diag_01_max']
rhum_feature_5_diag_00 = ['rhum_neigh_5_diag_00_mean', 'rhum_neigh_5_diag_00_min', 'rhum_neigh_5_diag_00_max']
rhum_feature_5_diag_pos = ['rhum_neigh_5_diag_pos_mean', 'rhum_neigh_5_diag_pos_min', 'rhum_neigh_5_diag_pos_max']

rhum_feature_5_lat = ['rhum_neigh_5_lat_mean', 'rhum_neigh_5_lat_min', 'rhum_neigh_5_lat_max']
rhum_feature_5_lon = ['rhum_neigh_5_lon_mean', 'rhum_neigh_5_lon_min', 'rhum_neigh_5_lon_max']
rhum_feature_9_lon = ['rhum_neigh_9_lon_mean', 'rhum_neigh_9_lon_min', 'rhum_neigh_9_lon_max']
rhum_feature_11_lon = ['rhum_neigh_11_lon_mean', 'rhum_neigh_11_lon_min', 'rhum_neigh_11_lon_max']
rhum_feature_15_lon = ['rhum_neigh_15_lon_mean', 'rhum_neigh_15_lon_min', 'rhum_neigh_15_lon_max']
rhum_feature_11_lat = ['rhum_neigh_11_lat_mean', 'rhum_neigh_11_lat_min', 'rhum_neigh_11_lat_max']
rhum_feature_9_lat = ['rhum_neigh_9_lat_mean', 'rhum_neigh_9_lat_min', 'rhum_neigh_9_lat_max']
rhum_feature_time_3 = ['rhum_time_3_mean', 'rhum_time_3_min', 'rhum_time_3_max']
rhum_feature_time_5 = ['rhum_time_5_mean', 'rhum_time_5_min', 'rhum_time_5_max']
rhum_feature_time_7 = ['rhum_time_7_mean', 'rhum_time_7_min', 'rhum_time_7_max']
rhum_feature_time_11 = ['rhum_time_11_mean', 'rhum_time_11_min', 'rhum_time_11_max']
rhum_feature_time_7_mid = ['rhum_time_7_mid_mean', 'rhum_time_7_mid_min', 'rhum_time_7_mid_max']
rhum_feature_time_14 = ['rhum_time_14_mean', 'rhum_time_14_min', 'rhum_time_14_max']
rhum_feature_time_21 = ['rhum_time_21_mean', 'rhum_time_21_min', 'rhum_time_21_max']
rhum_feature_time_28 = ['rhum_time_28_mean', 'rhum_time_28_min', 'rhum_time_28_max']
rhum_feature_time_9 = ['rhum_time_9_mean', 'rhum_time_9_min', 'rhum_time_9_max']

rhum_feature_3 = ['rhum_neigh_3_mean', 'rhum_neigh_3_min', 'rhum_neigh_3_max']
rhum_feature_7 = ['rhum_neigh_7_mean', 'rhum_neigh_7_min', 'rhum_neigh_7_max']
rhum_feature_9 = ['rhum_neigh_9_mean', 'rhum_neigh_9_min', 'rhum_neigh_9_max']
rhum_feature_11 = ['rhum_neigh_11_mean', 'rhum_neigh_11_min', 'rhum_neigh_11_max']
rhum_feature_13 = ['rhum_neigh_13_mean', 'rhum_neigh_13_min', 'rhum_neigh_13_max']


slp_feature_5 = ['slp_neigh_5_mean', 'slp_neigh_5_min', 'slp_neigh_5_max']
slp_feature_5_diag_11 = ['slp_neigh_5_diag_11_mean', 'slp_neigh_5_diag_11_min', 'slp_neigh_5_diag_11_max']
slp_feature_5_diag_10 = ['slp_neigh_5_diag_10_mean', 'slp_neigh_5_diag_10_min', 'slp_neigh_5_diag_10_max']
slp_feature_5_diag_01 = ['slp_neigh_5_diag_01_mean', 'slp_neigh_5_diag_01_min', 'slp_neigh_5_diag_01_max']
slp_feature_5_diag_00 = ['slp_neigh_5_diag_00_mean', 'slp_neigh_5_diag_00_min', 'slp_neigh_5_diag_00_max']
slp_feature_5_diag_pos = ['slp_neigh_5_diag_pos_mean', 'slp_neigh_5_diag_pos_min', 'slp_neigh_5_diag_pos_max']
slp_feature_5_lat = ['slp_neigh_5_lat_mean', 'slp_neigh_5_lat_min', 'slp_neigh_5_lat_max']
slp_feature_5_lon = ['slp_neigh_5_lon_mean', 'slp_neigh_5_lon_min', 'slp_neigh_5_lon_max']
slp_feature_9_lon = ['slp_neigh_9_lon_mean', 'slp_neigh_9_lon_min', 'slp_neigh_9_lon_max']
slp_feature_11_lon = ['slp_neigh_11_lon_mean', 'slp_neigh_11_lon_min', 'slp_neigh_11_lon_max']
slp_feature_15_lon = ['slp_neigh_15_lon_mean', 'slp_neigh_15_lon_min', 'slp_neigh_15_lon_max']
slp_feature_11_lat = ['slp_neigh_11_lat_mean', 'slp_neigh_11_lat_min', 'slp_neigh_11_lat_max']
slp_feature_9_lat = ['slp_neigh_9_lat_mean', 'slp_neigh_9_lat_min', 'slp_neigh_9_lat_max']
slp_feature_time_5 = ['slp_time_5_mean', 'slp_time_5_min', 'slp_time_5_max']
slp_feature_time_7 = ['slp_time_7_mean', 'slp_time_7_min', 'slp_time_7_max']
slp_feature_time_11 = ['slp_time_11_mean', 'slp_time_11_min', 'slp_time_11_max']
slp_feature_time_14 = ['slp_time_14_mean', 'slp_time_14_min', 'slp_time_14_max']
slp_feature_time_9 = ['slp_time_9_mean', 'slp_time_9_min', 'slp_time_9_max']

slp_feature_3 = ['slp_neigh_3_mean', 'slp_neigh_3_min', 'slp_neigh_3_max']
slp_feature_7 = ['slp_neigh_7_mean', 'slp_neigh_7_min', 'slp_neigh_7_max']
slp_feature_9 = ['slp_neigh_9_mean', 'slp_neigh_9_min', 'slp_neigh_9_max']
slp_feature_11 = ['slp_neigh_11_mean', 'slp_neigh_11_min', 'slp_neigh_11_max']
slp_feature_13 = ['slp_neigh_13_mean', 'slp_neigh_13_min', 'slp_neigh_13_max']


precip_feature_5 = ['precip_neigh_5_mean', 'precip_neigh_5_min', 'precip_neigh_5_max']
precip_feature_5_diag_11 = ['precip_neigh_5_diag_11_mean', 'precip_neigh_5_diag_11_min', 'precip_neigh_5_diag_11_max']
precip_feature_5_diag_10 = ['precip_neigh_5_diag_10_mean', 'precip_neigh_5_diag_10_min', 'precip_neigh_5_diag_10_max']
precip_feature_5_diag_01 = ['precip_neigh_5_diag_01_mean', 'precip_neigh_5_diag_01_min', 'precip_neigh_5_diag_01_max']
precip_feature_5_diag_00 = ['precip_neigh_5_diag_00_mean', 'precip_neigh_5_diag_00_min', 'precip_neigh_5_diag_00_max']
precip_feature_5_diag_pos = ['precip_neigh_5_diag_pos_mean', 'precip_neigh_5_diag_pos_min', 'precip_neigh_5_diag_pos_max']
precip_feature_5_lat = ['precip_neigh_5_lat_mean', 'precip_neigh_5_lat_min', 'precip_neigh_5_lat_max']
precip_feature_5_lon = ['precip_neigh_5_lon_mean', 'precip_neigh_5_lon_min', 'precip_neigh_5_lon_max']
precip_feature_9_lon = ['precip_neigh_9_lon_mean', 'precip_neigh_9_lon_min', 'precip_neigh_9_lon_max']
precip_feature_11_lon = ['precip_neigh_11_lon_mean', 'precip_neigh_11_lon_min', 'precip_neigh_11_lon_max']
precip_feature_15_lon = ['precip_neigh_15_lon_mean', 'precip_neigh_15_lon_min', 'precip_neigh_15_lon_max']
precip_feature_11_lat = ['precip_neigh_11_lat_mean', 'precip_neigh_11_lat_min', 'precip_neigh_11_lat_max']
precip_feature_9_lat = ['precip_neigh_9_lat_mean', 'precip_neigh_9_lat_min', 'precip_neigh_9_lat_max']
precip_feature_time_3 = ['precip_time_3_mean', 'precip_time_3_min', 'precip_time_3_max']
precip_feature_time_5 = ['precip_time_5_mean', 'precip_time_5_min', 'precip_time_5_max']
precip_feature_time_7 = ['precip_time_7_mean', 'precip_time_7_min', 'precip_time_7_max']
precip_feature_time_11 = ['precip_time_11_mean', 'precip_time_11_min', 'precip_time_11_max']
precip_feature_time_7_mid = ['precip_time_7_mid_mean', 'precip_time_7_mid_min', 'precip_time_7_mid_max']
precip_feature_time_9 = ['precip_time_9_mean', 'precip_time_9_min', 'precip_time_9_max']
precip_feature_time_14 = ['precip_time_14_mean', 'precip_time_14_min', 'precip_time_14_max']
precip_feature_time_21 = ['precip_time_21_mean', 'precip_time_21_min', 'precip_time_21_max']
precip_feature_time_28 = ['precip_time_28_mean', 'precip_time_28_min', 'precip_time_28_max']

precip_feature_3 = ['precip_neigh_3_mean', 'precip_neigh_3_min', 'precip_neigh_3_max']
precip_feature_7 = ['precip_neigh_7_mean', 'precip_neigh_7_min', 'precip_neigh_7_max']
precip_feature_9 = ['precip_neigh_9_mean', 'precip_neigh_9_min', 'precip_neigh_9_max']
precip_feature_11 = ['precip_neigh_11_mean', 'precip_neigh_11_min', 'precip_neigh_11_max']
precip_feature_13 = ['precip_neigh_13_mean', 'precip_neigh_13_min', 'precip_neigh_13_max']


pres_feature_5 = ['pres_neigh_5_mean', 'pres_neigh_5_min', 'pres_neigh_5_max']
pres_feature_5_diag_11 = ['pres_neigh_5_diag_11_mean', 'pres_neigh_5_diag_11_min', 'pres_neigh_5_diag_11_max']
pres_feature_5_diag_10 = ['pres_neigh_5_diag_10_mean', 'pres_neigh_5_diag_10_min', 'pres_neigh_5_diag_10_max']
pres_feature_5_diag_01 = ['pres_neigh_5_diag_01_mean', 'pres_neigh_5_diag_01_min', 'pres_neigh_5_diag_01_max']
pres_feature_5_diag_00 = ['pres_neigh_5_diag_00_mean', 'pres_neigh_5_diag_00_min', 'pres_neigh_5_diag_00_max']
pres_feature_5_diag_pos = ['pres_neigh_5_diag_pos_mean', 'pres_neigh_5_diag_pos_min', 'pres_neigh_5_diag_pos_max']
pres_feature_5_lat = ['pres_neigh_5_lat_mean', 'pres_neigh_5_lat_min', 'pres_neigh_5_lat_max']
pres_feature_5_lon = ['pres_neigh_5_lon_mean', 'pres_neigh_5_lon_min', 'pres_neigh_5_lon_max']
pres_feature_9_lon = ['pres_neigh_9_lon_mean', 'pres_neigh_9_lon_min', 'pres_neigh_9_lon_max']
pres_feature_15_lon = ['pres_neigh_15_lon_mean', 'pres_neigh_15_lon_min', 'pres_neigh_15_lon_max']
pres_feature_11_lat = ['pres_neigh_11_lat_mean', 'pres_neigh_11_lat_min', 'pres_neigh_11_lat_max']
pres_feature_9_lat = ['pres_neigh_9_lat_mean', 'pres_neigh_9_lat_min', 'pres_neigh_9_lat_max']
pres_feature_time_3 = ['pres_time_3_mean', 'pres_time_3_min', 'pres_time_3_max']
pres_feature_time_5 = ['pres_time_5_mean', 'pres_time_5_min', 'pres_time_5_max']
pres_feature_time_7 = ['pres_time_7_mean', 'pres_time_7_min', 'pres_time_7_max']
pres_feature_time_11 = ['pres_time_11_mean', 'pres_time_11_min', 'pres_time_11_max']
pres_feature_time_7_mid = ['pres_time_7_mid_mean', 'pres_time_7_mid_min', 'pres_time_7_mid_max']
pres_feature_time_9 = ['pres_time_9_mean', 'pres_time_9_min', 'pres_time_9_max']
pres_feature_time_14 = ['pres_time_14_mean', 'pres_time_14_min', 'pres_time_14_max']
pres_feature_time_21 = ['pres_time_21_mean', 'pres_time_21_min', 'pres_time_21_max']
pres_feature_time_28 = ['pres_time_28_mean', 'pres_time_28_min', 'pres_time_28_max']

pres_feature_3 = ['pres_neigh_3_mean', 'pres_neigh_3_min', 'pres_neigh_3_max']
pres_feature_7 = ['pres_neigh_7_mean', 'pres_neigh_7_min', 'pres_neigh_7_max']
pres_feature_9 = ['pres_neigh_9_mean', 'pres_neigh_9_min', 'pres_neigh_9_max']
pres_feature_11 = ['pres_neigh_11_mean', 'pres_neigh_11_min', 'pres_neigh_11_max']
pres_feature_13 = ['pres_neigh_13_mean', 'pres_neigh_13_min', 'pres_neigh_13_max']



wind_feature_11 = ['wind_500_neigh_11_mean', 'wind_500_neigh_11_min', 'wind_500_neigh_11_max']
wind_feature_9 = ['wind_500_neigh_9_mean', 'wind_500_neigh_9_min', 'wind_500_neigh_9_max']
wind_feature_5 = ['wind_500_neigh_5_mean', 'wind_500_neigh_5_min', 'wind_500_neigh_5_max']
wind_feature_5_diag_11 = ['wind_500_neigh_5_diag_11_mean', 'wind_500_neigh_5_diag_11_min', 'wind_500_neigh_5_diag_11_max']
wind_feature_5_diag_10 = ['wind_500_neigh_5_diag_10_mean', 'wind_500_neigh_5_diag_10_min', 'wind_500_neigh_5_diag_10_max']
wind_feature_5_diag_01 = ['wind_500_neigh_5_diag_01_mean', 'wind_500_neigh_5_diag_01_min', 'wind_500_neigh_5_diag_01_max']
wind_feature_5_diag_00 = ['wind_500_neigh_5_diag_00_mean', 'wind_500_neigh_5_diag_00_min', 'wind_500_neigh_5_diag_00_max']
wind_feature_5_diag_pos = ['wind_500_neigh_5_diag_pos_mean', 'wind_500_neigh_5_diag_pos_min', 'wind_500_neigh_5_diag_pos_max']
wind_feature_5_lat = ['wind_500_neigh_5_lat_mean', 'wind_500_neigh_5_lat_min', 'wind_500_neigh_5_lat_max']
wind_feature_5_lon = ['wind_500_neigh_5_lon_mean', 'wind_500_neigh_5_lon_min', 'wind_500_neigh_5_lon_max']
wind_feature_9_lon = ['wind_500_neigh_9_lon_mean', 'wind_500_neigh_9_lon_min', 'wind_500_neigh_9_lon_max']
wind_feature_11_lon = ['wind_500_neigh_11_lon_mean', 'wind_500_neigh_11_lon_min', 'wind_500_neigh_11_lon_max']
wind_feature_15_lon = ['wind_500_neigh_15_lon_mean', 'wind_500_neigh_15_lon_min', 'wind_500_neigh_15_lon_max']
wind_feature_11_lat = ['wind_500_neigh_11_lat_mean', 'wind_500_neigh_11_lat_min', 'wind_500_neigh_11_lat_max']
wind_feature_9_lat = ['wind_500_neigh_9_lat_mean', 'wind_500_neigh_9_lat_min', 'wind_500_neigh_9_lat_max']
wind_feature_time_5 = ['wind_500_time_5_mean', 'wind_500_time_5_min', 'wind_500_time_5_max']
wind_feature_time_7 = ['wind_500_time_7_mean', 'wind_500_time_7_min', 'wind_500_time_7_max']
wind_feature_time_14 = ['wind_500_time_14_mean', 'wind_500_time_14_min', 'wind_500_time_14_max']
wind_feature_time_9 = ['wind_500_time_9_mean', 'wind_500_time_9_min', 'wind_500_time_9_max']
wind_feature_time_11 = ['wind_500_time_11_mean', 'wind_500_time_11_min', 'wind_500_time_11_max']
wind_feature_7 = ['wind_500_neigh_7_mean', 'wind_500_neigh_7_min', 'wind_500_neigh_7_max']
wind_feature_3 = ['wind_500_neigh_3_mean', 'wind_500_neigh_3_min', 'wind_500_neigh_3_max']
wind_feature_13 = ['wind_500_neigh_13_mean', 'wind_500_neigh_13_min', 'wind_500_neigh_13_max']

In [ ]:
# df1 = pd.read_csv('df_fe_5_lon_neg_neigh.csv')
# df2 = pd.read_csv('df_fe_5_lon_pos_neigh.csv')
# df3 = pd.read_csv('df_fe_5_lat_pos_neigh.csv')
# df4 = pd.read_csv('df_fe_5_lat_neg_neigh.csv')
# cols4 = sst_features_5_lat_neg + wind_feature_5_lat_neg + slp_feature_5_lat_neg + precip_feature_5_lat_neg + pres_feature_5_lat_neg + rhum_feature_5_lat_neg
# df[cols4] = df4[cols4]

# cols1 = sst_features_5_lon_neg + wind_feature_5_lon_neg + slp_feature_5_lon_neg + precip_feature_5_lon_neg + pres_feature_5_lon_neg + rhum_feature_5_lon_neg
# df[cols1] = df1[cols1]

# cols2 = sst_features_5_lon_pos + wind_feature_5_lon_pos + slp_feature_5_lon_pos + precip_feature_5_lon_pos + pres_feature_5_lon_pos + rhum_feature_5_lon_pos 
# df[cols2] = df2[cols2]

# cols3 = sst_features_5_lat_pos + wind_feature_5_lat_pos + slp_feature_5_lat_pos + precip_feature_5_lat_pos + pres_feature_5_lat_pos + rhum_feature_5_lat_pos
# df[cols3] = df3[cols3]

# del df1, df2, df3, df4


In [ ]:
# df1 = pd.read_csv('df_fe_5_diag_11_neigh.csv')
# df2 = pd.read_csv('df_fe_5_diag_01_neigh.csv')
# df3 = pd.read_csv('df_fe_5_diag_10_neigh.csv')
# df4 = pd.read_csv('df_fe_5_diag_00_neigh.csv')
# cols4 = sst_features_5_diag_00 + wind_feature_5_diag_00 + slp_feature_5_diag_00 + precip_feature_5_diag_00 + pres_feature_5_diag_00 + rhum_feature_5_diag_00
# df[cols4] = df4[cols4]

# cols1 = sst_features_5_diag_11 + wind_feature_5_diag_11 + slp_feature_5_diag_11 + precip_feature_5_diag_11 + pres_feature_5_diag_11 + rhum_feature_5_diag_11
# df[cols1] = df1[cols1]

# cols2 = sst_features_5_diag_01 + wind_feature_5_diag_01 + slp_feature_5_diag_01 + precip_feature_5_diag_01 + pres_feature_5_diag_01 + rhum_feature_5_diag_01 
# df[cols2] = df2[cols2]

# cols3 = sst_features_5_diag_10 + wind_feature_5_diag_10 + slp_feature_5_diag_10 + precip_feature_5_diag_10 + pres_feature_5_diag_10 + rhum_feature_5_diag_10
# df[cols3] = df3[cols3]

# del df1, df2, df3, df4

In [ ]:
# more_cols = ['2_neigh_5_mean',
#  '2_neigh_5_min',
#  '2_neigh_5_max',
#  '5_neigh_5_mean',
#  '5_neigh_5_min',
#  '5_neigh_5_max',
#  '7_neigh_5_mean',
#  '7_neigh_5_min',
#  '7_neigh_5_max',
#  '8_neigh_5_mean',
#  '8_neigh_5_min',
#  '8_neigh_5_max',
#  '9_neigh_5_mean',
#  '9_neigh_5_min',
#  '9_neigh_5_max',
#  '12_neigh_5_mean',
#  '12_neigh_5_min',
#  '12_neigh_5_max',
#  '19_neigh_5_mean',
#  '19_neigh_5_min',
#  '19_neigh_5_max']

In [17]:
add_cols = slp_feature_5_diag_pos + precip_feature_5_diag_pos + pres_feature_5_diag_pos + rhum_feature_5_diag_pos
add_df = pd.read_csv('./df_fe_5_diag_pos_neigh.csv')
df[add_cols] = add_df[add_cols]


In [18]:
train_df = df[:375734]
test_df = df[375734:]

In [57]:
selected_cols = features + sst_features_5 + wind_feature_5 + slp_feature_5 + precip_feature_5 + pres_feature_5 + rhum_feature_5 + slp_feature_3 + precip_feature_3 + pres_feature_3  + slp_feature_7 + rhum_feature_7 + sst_features_7 + precip_feature_7 + pres_feature_7 + wind_feature_7 + slp_feature_9 + wind_feature_9 + pres_feature_9 + precip_feature_9 + rhum_feature_9 + rhum_feature_13 + \
rhum_feature_time_7 + pres_feature_time_7 + precip_feature_time_7 + \
rhum_feature_time_14 + pres_feature_time_14 + precip_feature_time_14 + \
sst_features_5_lat + wind_feature_5_lat + slp_feature_5_lat + precip_feature_5_lat + pres_feature_5_lat + rhum_feature_5_lat + \
sst_features_5_lon + wind_feature_5_lon + slp_feature_5_lon + precip_feature_5_lon + pres_feature_5_lon + rhum_feature_5_lon + \
slp_feature_5_diag_pos + precip_feature_5_diag_pos + pres_feature_5_diag_pos + rhum_feature_5_diag_pos + \
rhum_feature_time_3_pas[:1] + pres_feature_time_3_pas[:1] + precip_feature_time_3_pas[:1]


In [58]:
cols = selected_cols
len(cols)

305

In [59]:
train_df['loc_group'] = train_df['loc_group'].astype('category')
test_df['loc_group'] = test_df['loc_group'].astype('category')

/tmp/ipykernel_3500999/3521624167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['loc_group'] = train_df['loc_group'].astype('category')
/tmp/ipykernel_3500999/3521624167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['loc_group'] = test_df['loc_group'].astype('category')


In [60]:

def train_test_by_df(train_df_, test_df_, iterations):
  X = train_df_[cols].values
  y = train_df_[target[0]].values
    

  skf = KFold(n_splits=5, random_state=2048, shuffle=True)
  skf.get_n_splits(X, y)

  train_index_list = []
  test_index_list = []
  train_x_list = []
  val_x_list = []
  train_y_list = []
  val_y_list = []

  for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index]) 
  
    
  cat_models = []
  for i in tqdm(range(5)):
    train_x, val_x = train_x_list[i], val_x_list[i]
    train_y, val_y = train_y_list[i], val_y_list[i]
    clf = CatBoostRegressor( iterations = iterations, verbose = 1000, task_type = 'GPU',  devices = '0')
    clf.fit(train_x, train_y, eval_set=[(val_x, val_y)])
    cat_models.append(clf)

  preds = []
  for cat_model in tqdm(cat_models):
      preds.append(cat_model.predict(test_df_[cols].values))
  s = 0
  for pred in preds:
      s += pred
  s = s/5
  
  return s, cat_models


In [61]:
s, cat_models = train_test_by_df(train_df, test_df, 5000)

5it [00:58, 11.76s/it]
  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 2495.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2556777	test: 9.2879362	best: 9.2879362 (0)	total: 24.2ms	remaining: 2m 1s
1000:	learn: 0.5502891	test: 0.5654986	best: 0.5654986 (1000)	total: 22.6s	remaining: 1m 30s
2000:	learn: 0.4161743	test: 0.4371218	best: 0.4371218 (2000)	total: 46.2s	remaining: 1m 9s
3000:	learn: 0.3505066	test: 0.3759462	best: 0.3759462 (3000)	total: 1m 8s	remaining: 45.6s
4000:	learn: 0.3084848	test: 0.3375350	best: 0.3375350 (4000)	total: 1m 30s	remaining: 22.7s
4999:	learn: 0.2786574	test: 0.3109402	best: 0.3109402 (4999)	total: 1m 52s	remaining: 0us
bestTest = 0.3109401567
bestIteration = 4999


 20%|█████████████████▍                                                                     | 1/5 [02:25<09:43, 145.91s/it]Warning: less than 75% gpu memory available for training. Free: 2495.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2666254	test: 9.2596282	best: 9.2596282 (0)	total: 10.4ms	remaining: 52.1s
1000:	learn: 0.5500780	test: 0.5652756	best: 0.5652756 (1000)	total: 22.7s	remaining: 1m 30s
2000:	learn: 0.4150048	test: 0.4365650	best: 0.4365650 (2000)	total: 45.2s	remaining: 1m 7s
3000:	learn: 0.3492402	test: 0.3753111	best: 0.3753111 (3000)	total: 1m 6s	remaining: 44.4s
4000:	learn: 0.3072138	test: 0.3371549	best: 0.3371549 (4000)	total: 1m 28s	remaining: 22.1s
4999:	learn: 0.2774762	test: 0.3109381	best: 0.3109381 (4999)	total: 1m 50s	remaining: 0us
bestTest = 0.3109381401
bestIteration = 4999


 40%|██████████████████████████████████▊                                                    | 2/5 [04:50<07:15, 145.05s/it]Warning: less than 75% gpu memory available for training. Free: 2495.5625 Total: 24268.3125


...

 60%|████████████████████████████████████████████████████▏                                  | 3/5 [07:15<04:50, 145.18s/it]Warning: less than 75% gpu memory available for training. Free: 2495.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2624373	test: 9.2764539	best: 9.2764539 (0)	total: 32.9ms	remaining: 2m 44s
1000:	learn: 0.5531529	test: 0.5648840	best: 0.5648840 (1000)	total: 22.2s	remaining: 1m 28s
2000:	learn: 0.4169138	test: 0.4360063	best: 0.4360063 (2000)	total: 44.8s	remaining: 1m 7s
3000:	learn: 0.3511055	test: 0.3755246	best: 0.3755246 (3000)	total: 1m 6s	remaining: 44.6s
4000:	learn: 0.3098757	test: 0.3386510	best: 0.3386510 (4000)	total: 1m 28s	remaining: 22.2s
4999:	learn: 0.2796571	test: 0.3117934	best: 0.3117934 (4999)	total: 1m 50s	remaining: 0us
bestTest = 0.3117934387
bestIteration = 4999


 80%|█████████████████████████████████████████████████████████████████████▌                 | 4/5 [09:41<02:25, 145.29s/it]Warning: less than 75% gpu memory available for training. Free: 2495.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2688503	test: 9.2383169	best: 9.2383169 (0)	total: 21.5ms	remaining: 1m 47s
1000:	learn: 0.5531867	test: 0.5667106	best: 0.5667106 (1000)	total: 21.9s	remaining: 1m 27s
3000:	learn: 0.3512395	test: 0.3758792	best: 0.3758792 (3000)	total: 1m 7s	remaining: 44.8s
4000:	learn: 0.3092982	test: 0.3378411	best: 0.3378411 (4000)	total: 1m 30s	remaining: 22.6s
4999:	learn: 0.2792864	test: 0.3111376	best: 0.3111376 (4999)	total: 1m 55s	remaining: 0us
bestTest = 0.3111375723
bestIteration = 4999


100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.03s/it]


In [62]:
output_df = pd.DataFrame(columns = ['index', 'pred'])
output_df['index'] = [int(i) for i in test_df['index']]
output_df['pred'] = s
res_df = pd.read_csv('./sample_solution.csv')
res_df = res_df.merge(output_df, how='left', on = ['index'])
res_df['contest-tmp2m-14d__tmp2m'] = res_df['pred']
res_df[[target[0], 'index']].to_csv('submission_timeseries_3.csv', index=False)

In [63]:
# best_df = pd.read_csv('./best_sub_cat.csv')
# t = best_df[target[0]] * ((1 + 0.1 / 30))
# best_df[target[0]] = t
# best_df.to_csv('submission_post.csv', index=False)

In [64]:
# res_df = pd.read_csv('./sample_solution.csv')
# cat_df = pd.read_csv('./submission_fe_pos_v2.csv')
# lgb_df = pd.read_csv('./best_lgb.csv')
# xgb_df = pd.read_csv('./best_xgb.csv')
# s = cat_df[target[0]] * 0.73 + lgb_df[target[0]] * 0.25 + xgb_df[target[0]] * 0.02
# res_df[target[0]] = s
# res_df[[target[0], 'index']].to_csv('submission_ensemble.csv', index=False)